# Demo 02 - Adaptive Spectrum Processing

This notebook presents an example to read in a sweep of raw data and perform the Adaptive Spectrum Processing method, which was published by Kong++12. The original algorithm description is documented in the paper:

\[Kong++12\] Kong, F., Y. Zhang, R. Palmer, "Wind Turbine Clutter Mitigation for Weather Radar by Adaptive Spectrum Processing," __2012 IEEE Radar Conference__, Atlanta, GA, 2012.

The process can be summarized as follows:

 - Contaminated cells are hand-picked where it has been indicated to contain wind turbines.
 - The clean area is derived from expanding the contaminated area using dilation.
 - Average velocity distribution is derived using fft(), incoherently averaged.
 - Newly added for practicality: a further smoothing is accomplished using a rolling average, m-tap.

In [ ]:
import os
import time
import glob
import numpy as np
import scipy as sp
import scipy.signal
import matplotlib
import matplotlib.patheffects
import matplotlib.pyplot as plt
import random

import toshi
import cspec

plt.style.use('./darkmode.style')
zmap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', cspec.colormap.zmap()[:, :3])
vmap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', cspec.colormap.vmap()[:, :3], 64)
imap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', cspec.colormap.c12map()[:, :3], 12)

path_effects = [
    matplotlib.patheffects.Stroke(linewidth=1.6, foreground=(0, 0, 0, 0.8)),
    matplotlib.patheffects.Normal()
]

In [ ]:
waveform = toshi.tx_waveform()
pc_nfft = 1024
wf = sp.fft.fft(waveform / np.sqrt(np.sum(np.abs(waveform) ** 2)), n=pc_nfft)

wtc_loc = toshi.wtc_loc_from_csv()
turbines = np.array([[pt[2], pt[1]] for pt in wtc_loc], dtype=np.single)

In [ ]:
data_home = os.path.expanduser('~/Downloads/toshiba/AKITA_IQ_TO_OU/IQdata')
files = glob.glob('{}/**/*.dat'.format(data_home), recursive=True)
files = sorted(files)

In [ ]:
# file = '/Volumes/Transcend/akita/IQ/IQdata/2020.06.16/07/20200616_071735.439598-9E-02.iqData.XXXX.AKITA.dat'
# file = os.path.expanduser('~/Downloads/toshiba/AKITA_IQ_TO_OU/IQdata/2021.01.28/12/20210128_124008.095007-CA-68.iqData.XXXX.AKITA.dat')

file = files[8]
filesize = os.path.getsize(file)
print('{} - {:,.0f} B'.format(file, filesize))

In [ ]:
s = time.time()
all_ray_pulses, all_cpi_headers = toshi.read(file)
e = time.time()
print('Data read in {:.2f} s'.format(e - s))

In [ ]:
# Go through the pulses for azimuth
az = np.zeros(len(all_ray_pulses), dtype=np.single)
for k, pulses in enumerate(all_ray_pulses):
    az[k] = pulses[0].azimuth
    
# Choose ray 2 to whatever that completes the 360-deg coverage
n = np.argmin(np.abs(az[3:] - az[2])) + 1
az = az[2:n+2]
ray_pulses = all_ray_pulses[2:n+2]
cpi_headers = all_cpi_headers[2:n+2]

# Dimensions
naz = len(ray_pulses)
ngate_long = ray_pulses[0][0].cpi_header.num_range_long_hi
ngate_short_hi = ray_pulses[0][0].cpi_header.num_range_short_hi
ngate_short_lo = ray_pulses[0][0].cpi_header.num_range_short_lo
ngate = ngate_long + ngate_short_hi + ngate_short_lo

# Elevation assumed to be flat from the very first pulse
scan_el = ray_pulses[0][0].elevation
scan_time = time.strptime(os.path.basename(file)[:15], '%Y%m%d_%H%M%S')

# Sampling code from the CPI header
fs = 1.0e6 * (1 << cpi_headers[0].fs_code)
dr = 3.0e8 / fs / 2
r = 1.0e-3 * (np.arange(0, ngate, dtype=np.single) * dr + 0.5 * dr)

print('fs = {:,.0f} Hz'.format(fs))
print('dr = {:.1f} m'.format(dr))
print('n = {} -> 2 ... {} ({})'.format(n, n+2, naz))

In [ ]:
# Noise estimate, try azimuth 0, around 20-25 km
# ia = np.argmin(np.abs(a[3:] - 0.0)) + 1
# ir, er = np.argmin(np.abs(r - 20.0)), np.argmin(np.abs(r - 25.0))
# samples = np.zeros((len(ray_pulses[ia]), er-ir), dtype=np.csingle)

# Gather the samples. Ignore phase code since we are only interested in amplitude
# for k, pulse in enumerate(ray_pulses[ia]):
#     samples[k, :] = pulse.h_long_hi[ir:er]
# noise = np.mean(np.abs(samples)) ** 2

# Hard code noise estimate to be about 24 (eye ball)
noise = 24
print('Using noise estimate in 16-bit ADU: {:.4f}'.format(noise))

In [ ]:
s = np.zeros((naz, ngate), dtype=np.single)
v = np.zeros((naz, ngate), dtype=np.single)

nfft = 256
use_window = True
use_pulse_pair = False
use_dc_removal = False

# Go through the pulses
a = np.zeros(len(ray_pulses), dtype=np.single)
cpuls = []
for k, (pulses, cpi_header) in enumerate(zip(ray_pulses, cpi_headers)):
    a[k] = pulses[0].azimuth
    npulse = len(pulses)

    # Decode the long pulse, then compress using wf
    p = np.zeros((npulse, ngate_long), dtype=np.csingle)
    for j, pulse in enumerate(pulses):
        p[j, :] = pulse.h_long_hi * np.exp(-1j * pulse.phase_h_long)       # Phase decoding
    pf = sp.fft.fft(p, n=pc_nfft, axis=1)                                  # Pulse compression in Fourier domain
    pc = sp.fft.ifft(pf * wf, n=pc_nfft, axis=1)                           # Return to time domain

    # Gather the short pulses and the compressed long pulses
    p = np.zeros((npulse, ngate), dtype=np.csingle)
    for j, pulse in enumerate(pulses):
        c = np.exp(-1j * pulse.phase_h_short)                              # Phase code
        p[j, :ngate_long] = pc[j, :ngate_long]                             # Long only
        p[j, :ngate_short_hi] = pulse.h_short_hi * c                       # Short hi
        p[j, :ngate_short_lo] = pulse.h_short_lo * c                       # Short lo        

    if use_dc_removal:
        p = p - np.mean(p, axis=0)                                         # Remove DC if desired

    cpuls.append(pc)                                                       # Keep a copy of the compressed pulse

    if use_window:
        w = scipy.signal.get_window('blackmanharris', len(p))
    else:
        w = np.ones((p.shape[0],))
    w /= np.sqrt(np.sum(w ** 2)) / np.sqrt(p.shape[0])                     # Normalize to non-windowed gain
    ww = np.repeat(np.expand_dims(w, axis=1), ngate, axis=1)               # Make same shape
    p *= ww                                                                # Windowing

    if use_pulse_pair:
        pp = p[1:, :] * np.conj(p[:-1, :])                                 # x(n) * x'(n-1)
        s[k, :] = np.mean(np.abs(p) ** 2, axis=0)                          # s(n) = E[x(n) * x'(n)]
        v[k, :] = np.angle(np.sum(pp, axis=0))                             # r(1) = E[x(n) * x'(n-1)]
    else:
        spec = np.fft.fft(p, nfft, axis=0)                                 # FFT
        s[k, :] = np.mean(np.abs(spec) ** 2, axis=0) / p.shape[0]          # Periodogram
        v[k, :] = np.angle(np.fft.ifft(spec * np.conj(spec), axis=0)[1])   # IFT -> ACF[1]

# Signal
s -= noise
s[s <= 0] = 1.0e-6                                    # Avoid log(0)
snr = 10 * np.log10(s / noise)                        # Signal-to-noise ratio in dB
z = 10 * np.log10(s * (r + 0.5e-3 * dr) ** 2) - 40    # Estimated ZCal = -40 
z[:, :ngate_short_hi] += 15                           # ~15-dB on the short waveform, perhaps?

# Thresholding at SNR = 0 dB
m = snr < -1
z[m] = np.nan
v[m] = np.nan

## Chart

In [ ]:
# Edge of range cells
wid_a = np.mean(sorted(np.diff(az))[int(0.3 * len(az)):int(0.6 * len(az))])
end_a = az[-1] + wid_a
if end_a >= 360.0:
    end_a -= 360.0
ae = np.append(az, end_a)
re = 1.0e-3 * np.arange(0, ngate + 1, dtype=np.single) * dr

# Radar cell locations
ce = np.cos(np.deg2rad(scan_el))
rr, aa = np.meshgrid(re, np.deg2rad(ae))
xx = rr * ce * np.sin(aa)
yy = rr * ce * np.cos(aa)

# Rings
ring_r = np.arange(5, 21, 5)
ring_a = np.arange(0, 361, 2) / 180 * np.pi
ring_x = np.outer(np.sin(ring_a), ring_r)
ring_y = np.outer(np.cos(ring_a), ring_r)

# Turbine locations
a_turb = turbines[:, 0] / 180.0 * np.pi
r_turb = turbines[:, 1]
x_turb = r_turb * np.sin(a_turb)
y_turb = r_turb * np.cos(a_turb)

# Various domain to choose from
# xlim, ylim = (-3, 2), (2, 6.5)
# xlim, ylim = (-7, 6.5), (-6, 7)
# xlim, ylim = (-13, 5), (3, 18)
# xlim, ylim = (-25, 10), (0, 35)
xlim, ylim = (-18, 18), (-18, 18)

# Reflectivity plot
plt.figure(figsize=(5, 4), dpi=216)
plt.pcolormesh(xx[:, :501], yy[:, :501], z[:, :500], cmap=zmap)
plt.plot(x_turb, y_turb, 'xk', markersize=4)
plt.plot(ring_x, ring_y, 'k:', linewidth=0.75)
for i in range(len(ring_r) - 1):
    plt.text(ring_x[9, i], ring_y[9, i], '{} km'.format(ring_r[i]), 
             fontsize=6, fontweight='bold', path_effects=path_effects,
             ha='center', va='center', rotation=-17)
plt.clim((-32, 96))
plt.xlim(xlim)
plt.ylim(ylim)
plt.colorbar()
plt.grid(linewidth=0.25)
plt.title(os.path.basename(file), fontsize=8)

# Velocity plot
plt.figure(figsize=(5, 4), dpi=216)
plt.pcolormesh(xx[::2, :501], yy[::2, :501], v[::2, :500], cmap=vmap)
# plt.plot(x_turb, y_turb, 'xk', markersize=4)
plt.clim((-5, 5))
plt.xlim(xlim)
plt.ylim(ylim)
plt.colorbar()
plt.grid(linewidth=0.25)
plt.title(os.path.basename(file), fontsize=8)

plt.show()

## Wind Turbine Marking

In [ ]:
tags = cspec.wtc_tags(turbines, az, r)

In [ ]:
# Gather the 1st cluster
img = np.zeros(tags.shape)
for k in range(1, 5):
    mask = tags == k
    dmask = cspec.dilate(mask, n=2)
    gmask = dmask ^ mask

    # Cluster tags
    t = mask + 1.5 * gmask
    m = t > 0
    img[m] = t[m] + (k - 1)

img[img < 1] = np.nan
    
plt.figure(figsize=(5, 4), dpi=216)
plt.pcolormesh(xx, yy, img, cmap=imap, shading='flat')
# plt.pcolormesh(xx[::2, :500], yy[::2, :500], img[::2, :500], cmap=zmap)
plt.clim((0.75, 6.75))
# plt.pcolormesh(xx[::2, :500], yy[::2, :500], -5 * img[::2, :500] + v[::2, :500], cmap=vmap)
# plt.clim((-5, 5))
plt.xlim(xlim)
plt.ylim(ylim)
plt.colorbar()
plt.grid(linewidth=0.25)
plt.title('Contaminated and Clean Cells', fontsize=8)

plt.show()

In [ ]:
mask = tags == 1
dmask = cspec.dilate(mask, 2)
gmask = dmask ^ mask
gmask[snr < -1] = False

dirty_cells = cspec.mask2cells(mask)
clean_cells = cspec.mask2cells(gmask)

# Gather all clean cells, shuffle the order
g = 20
ii = np.arange(len(clean_cells))
random.shuffle(ii)
selected_cells = []
for ic in ii[:g]:
    selected_cells.append(clean_cells[ic])

selected_mask = cspec.cell2mask(mask.shape, selected_cells)

In [ ]:
# Show the clean cells
clean_mask = cspec.cell2mask(mask.shape, clean_cells)
clean_mask = np.array(clean_mask, dtype=np.single) + 0.5
clean_mask[clean_mask < 1] = np.nan

clean_mask[selected_mask] = 6

plt.figure(figsize=(5, 4), dpi=216)
plt.pcolormesh(xx, yy, clean_mask, cmap=imap, shading='flat')
plt.clim((0.75, 6.75))
plt.xlim(xlim)
plt.ylim(ylim)
plt.colorbar()
plt.grid(linewidth=0.25)
plt.title('Clean Cells and Selected Cells of Cluster #1', fontsize=8)
plt.show()

In [ ]:
filts = []
spec = np.zeros(nfft, dtype=np.single)
for (ia, ir) in selected_cells:
    q = cpuls[ia][:, ir]
    w = scipy.signal.get_window('blackmanharris', len(q))
    print((ia, ir), q.shape, w.shape)

    spec += np.abs(np.fft.fft(q * w, nfft))
    
# m-tap circular averaging
m = 3
filt = spec.copy()
for k in range(1, m):
    filt += np.roll(spec, k)
filt = np.roll(filt, -int(m/2))       # Compensate for running-average lag
filt /= m                             # Normalize by tap length
filt /= np.sqrt(np.mean(filt ** 2))   # Normalize by noise gain

# Add in a DC filter / GMAP-like zeroing
# m = int(nfft * 0.08)
# filt[:m] = 1.0e-2
# filt[-m:] = 1.0e-2

In [ ]:
omega = np.arange(-nfft/2, nfft/2) / nfft * np.pi

plt.figure(figsize=(8, 3), dpi=216)
# for k, g in enumerate(gs):
plt.plot(omega, 20.0 * np.log10(np.fft.fftshift(filt)), label='1')
plt.xlim((-1.6, 1.6))
plt.ylim((-45, 16))
plt.grid(linewidth=0.25)
plt.xlabel('Omega (rad/sample)')
plt.ylabel('Filter Amplitude (dB)')
plt.title('Adaptive Filter Derived from the Clean Cells (m = {})'.format(m))
# plt.legend(gs)
plt.show()

In [ ]:
# filts = []
# gs = [10, 20, 40, 80]

# for g in gs:
#     spec = np.zeros(nfft, dtype=np.single)
#     for (ia, ir) in clean_cells[:g]:
#         p = np.zeros(len(ray_pulses[ia]), dtype=np.csingle)
#         for k, pulse in enumerate(ray_pulses[ia]):
#             p[k] = pulse.h_short_hi[ir] * np.exp(-1j * pulse.phase_h_long)
#         #p -= np.mean(p)
#         w = scipy.signal.get_window('blackmanharris', len(p)).reshape((p.shape[0],))
#         spec += np.abs(np.fft.fft(p * w, nfft))
    
#     # m-tap circular averaging
#     m = 3
#     filt = spec.copy()
#     for k in range(1, m):
#         filt += np.roll(spec, k)
#     filt = np.roll(filt, -int(m/2))    # Compensate for running-average lag
#     filt /= m
#     filt /= np.sqrt(np.mean(filt ** 2))   # Normalize by noise gain

#     # Add in a DC filter / GMAP-like zeroing
#     m = int(nfft * 0.08)
#     filt[:m] = 1.0e-2
#     filt[-m:] = 1.0e-2

#     filts.append(filt)

In [ ]:
omega = np.arange(-nfft/2, nfft/2) / nfft * np.pi

plt.figure(figsize=(8, 3), dpi=100)
for k, g in enumerate(gs):
    plt.plot(omega, 20.0 * np.log10(np.fft.fftshift(filts[k])), label=g)
plt.xlim((-1.6, 1.6))
plt.ylim((-45, 16))
plt.grid(linewidth=0.25)
plt.xlabel('Omega (rad/sample)')
plt.ylabel('Filter Amplitude (dB)')
plt.title('Adaptive Filter Derived from the Clean Cells (m = {})'.format(m))
plt.legend(gs)
plt.show()

In [ ]:
filt = filts[0]

sc = s.copy() + noise
vc = v.copy()
use_window = True

# Go through the cells
for (ia, ir) in dirty_cells:
    p = np.zeros(len(ray_pulses[ia]), dtype=np.csingle)
    a[k] = pulses[0].azimuth
    for k, pulse in enumerate(ray_pulses[ia]):
        p[k] = pulse.h_short_hi[ir] * np.exp(-1j * pulse.phase_h_short)
    p = p - np.mean(p, axis=0)                                  # Remove DC if desired
    # For sanity check, this should be identical to the original method
    if use_window:
        w = scipy.signal.get_window('blackmanharris', len(p)).reshape((p.shape[0],))
        w /= np.sqrt(np.mean(w ** 2))
        spec = np.abs(np.fft.fft(p * w, nfft))
    else:
        spec = np.abs(np.fft.fft(p, nfft))
    # Filtering in spectral domain
    spec *= filt
    sc[ia, ir] = np.sum(spec ** 2 / len(p) / nfft)
    vc[ia, ir] = np.angle(np.fft.ifft(spec * np.conj(spec), axis=0)[1])   # IFT -> ACF[1]

# Signal
sc -= noise
sc[sc <= 0] = 1.0e-6                                    # Avoid log(0)
snr = 10 * np.log10(sc / noise)                         # Signal-to-noise ratio in dB
zc = 10 * np.log10(sc * (r + 0.5e-3 * dr) ** 2) - 40    # Estimated ZCal = -40 
zc[:, :ngate_short] += 18                               # ~18-dB on the short waveform, perhaps?

# Thresholding
m = snr < 0
zc[m] = np.nan
vc[m] = np.nan

In [ ]:
prefix = os.path.expanduser('~/Downloads/{}'.format(os.path.basename(file)[:-3]))

In [ ]:
# Filtered Reflectivity plot
plt.figure(figsize=(5.2, 4), dpi=200)
plt.pcolormesh(xx[:, :500], yy[:, :500], zc[:, :500], cmap=zmap)
# plt.plot(x_turb, y_turb, 'xk', markersize=4)
plt.clim((-32, 96))
plt.xlim(xlim)
plt.ylim(ylim)
plt.colorbar()
plt.grid(linewidth=0.25)
plt.title(os.path.basename(file), fontsize=8)
# plt.savefig('{}Zc.png'.format(prefix))

# Filtered Velocity plot
plt.figure(figsize=(5.2, 4), dpi=200)
plt.pcolormesh(xx[::2, :500], yy[::2, :500], vc[::2, :500], cmap=vmap)
# plt.plot(x_turb, y_turb, 'xk', markersize=4)
plt.clim((-5, 5))
plt.xlim(xlim)
plt.ylim(ylim)
plt.colorbar()
plt.grid(linewidth=0.25)
plt.title(os.path.basename(file), fontsize=8)
# plt.savefig('{}Vc.png'.format(prefix))

plt.show()

In [ ]:
# Montage plot
fig, axs = plt.subplots(2, 2, figsize=(8, 5.5), dpi=200)

ax = axs[0, 0]
im = ax.pcolormesh(xx[:, :150], yy[:, :150], z[:, :150], cmap=zmap)
# ax.plot(x_turb, y_turb, 'xk', markersize=4)
im.set_clim((-32, 96))
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.grid(linewidth=0.25)
ax.set_title('Original Z', fontsize=9)
ax.set_xticklabels([])
fig.colorbar(im, ax=ax)

ax = axs[0, 1]
im = ax.pcolormesh(xx[::2, :150], yy[::2, :150], v[::2, :150], cmap=vmap)
# ax.plot(x_turb, y_turb, 'xk', markersize=4)
im.set_clim((-5, 5))
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.grid(linewidth=0.25)
ax.set_title('Original V', fontsize=9)
ax.set_xticklabels([])
fig.colorbar(im, ax=ax)

ax = axs[1, 0]
im = ax.pcolormesh(xx[:, :150], yy[:, :150], zc[:, :150], cmap=zmap)
im.set_clim((-32, 96))
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.grid(linewidth=0.25)
ax.set_title('Filtered Z', fontsize=9)
fig.colorbar(im, ax=ax)

ax = axs[1, 1]
im = ax.pcolormesh(xx[::2, :150], yy[::2, :150], vc[::2, :150], cmap=vmap)
im.set_clim((-5, 5))
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.grid(linewidth=0.25)
ax.set_title('Filtered V', fontsize=9)
fig.colorbar(im, ax=ax)

plt.suptitle(os.path.basename(file))
plt.savefig('{}png'.format(prefix))

In [ ]:
plt.close()